Para instalar la librería "randomForest" en Ubuntu, primero deberemos instalar Fortran ejecutando en la terminal:

`sudo apt install gfortran`

posteriormente abrimos R y ejecutamos:

`install.packages("randomForest")`

En caso de que no funcione la instalación anterior, vamos a la página oficial de librerías de R: 

https://cran.r-project.org/src/contrib/Archive/

Buscamos la librería "randomForest" y nos aparecerá una lista con diferentes versiones de la librería.

Copiamos el enlace de una versión específica, por ejemplo (la versión de 2022): 

https://cran.r-project.org/src/contrib/Archive/randomForest/randomForest_4.7-1.tar.gz

Posteriormente instalamos la librería como se muestra a continuación:

```
urlPackage <- "https://cran.r-project.org/src/contrib/Archive/randomForest/randomForest_4.7-1.tar.gz"
install.packages(urlPackage, repos=NULL, type="source")
```

In [2]:
# librería que contiene el dataset "kyphosis"
library(rpart)

# pquete para particionar datos para prueba y entrenamiento:
# install.packages('caTools')
library(caTools)

# install.packages("randomForest")
library(randomForest)

In [3]:
df <- kyphosis
class(df)

[1] "data.frame"

In [4]:
head(df)

,Kyphosis,Age,Number,Start
,<fct>,<int>,<int>,<int>
1,absent,71,3,5
2,absent,158,3,14
3,present,128,4,5
4,absent,2,5,1
5,absent,1,4,15
6,absent,1,2,16


In [5]:
str(df)

'data.frame':	81 obs. of  4 variables:
 $ Kyphosis: Factor w/ 2 levels "absent","present": 1 1 2 1 1 1 1 1 1 2 ...
 $ Age     : int  71 158 128 2 1 1 61 37 113 59 ...
 $ Number  : int  3 3 4 5 4 2 2 3 2 6 ...
 $ Start   : int  5 14 5 1 15 16 17 16 16 12 ...


## Particionamos datos para entrenamiento y pruebas

In [6]:
# establecemos una semilla para que se preserve la configuración del particionamiento de datos:
set.seed(80)
p_train <- 0.7 #  <-- Porcentaje para datos de entrenamiento
booleans <- sample.split( df$Kyphosis, SplitRatio = p_train )

In [9]:
df_train <- subset( df, booleans == TRUE)
head(df_train)

,Kyphosis,Age,Number,Start
,<fct>,<int>,<int>,<int>
1,absent,71,3,5
2,absent,158,3,14
5,absent,1,4,15
6,absent,1,2,16
7,absent,61,2,17
10,present,59,6,12


In [10]:
df_test <- subset( df, booleans == FALSE)
head(df_test)

,Kyphosis,Age,Number,Start
,<fct>,<int>,<int>,<int>
3,present,128,4,5
4,absent,2,5,1
8,absent,37,3,16
9,absent,113,2,16
13,absent,18,5,2
17,absent,78,6,15


## Entrenamos el modelo

In [11]:
# label --> "Kyphosis"

modelo <- randomForest(Kyphosis ~ . , df_train)

In [12]:
print(modelo)


Call:
 randomForest(formula = Kyphosis ~ ., data = df_train) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 1

        OOB estimate of  error rate: 24.56%
Confusion matrix:
        absent present class.error
absent      40       5   0.1111111
present      9       3   0.7500000


## Realizamos predicciones con los datos de prueba

In [13]:
approx <- predict(modelo, df_test)
approx

3       4       8       9      13      17      22      25      26      33 
present  absent  absent  absent  absent  absent present  absent  absent  absent 
     37      46      47      57      58      59      63      64      68      70 
 absent  absent  absent  absent present present  absent  absent  absent  absent 
     72      75      79      81 
 absent  absent  absent  absent 
Levels: absent present

In [14]:
df_test['predicciones'] <- approx

In [15]:
df_test

,Kyphosis,Age,Number,Start,predicciones
,<fct>,<int>,<int>,<int>,<fct>
3,present,128,4,5,present
4,absent,2,5,1,absent
8,absent,37,3,16,absent
9,absent,113,2,16,absent
13,absent,18,5,2,absent
17,absent,78,6,15,absent
22,present,105,6,5,present
25,present,15,7,2,absent
26,absent,9,5,13,absent


In [16]:
N <- nrow(df_test)
N

[1] 24

In [18]:
obs <- df_test$Kyphosis
aprox <- df_test$predicciones

In [24]:
k <- 0
for(i in 1:N){
    if( obs[i] == aprox[i]){
        k <- k + 1
    }
}

acc <- k/N
acc

[1] 0.875

In [27]:
texto <- paste('El modelo tiene un accuracy del',acc*100,'%')
print(texto)

[1] "El modelo tiene un accuracy del 87.5 %"
